In [1]:
import sys
import pickle
import pandas as pd
import numpy as np

In [2]:
filename = "co_T0_T1_toshare.pkl"
area_to_minority = "OAtoMinorityData.CSV"

In [3]:
pickle_in = open(filename,"rb")
df = pickle.load(pickle_in)

In [4]:
from IPython.display import display
pd.options.display.max_columns = None
df.head()

,CompanyName,CompanyNumber,RegAddressCareOf,RegAddressPOBox,RegAddressAddressLine1,RegAddressAddressLine2,RegAddressPostTown,RegAddressCounty,RegAddressCountry,RegAddressPostCode,CompanyCategory,CompanyStatus_T0,CountryOfOrigin,DissolutionDate,IncorporationDate,AccountsAccountRefDay,AccountsAccountRefMonth,AccountsNextDueDate,AccountsLastMadeUpDate,AccountsAccountCategory,ReturnsNextDueDate,ReturnsLastMadeUpDate,MortgagesNumMortCharges,MortgagesNumMortOutstanding,MortgagesNumMortPartSatisfied,MortgagesNumMortSatisfied,LimitedPartnershipsNumGenPartners,LimitedPartnershipsNumLimPartners,PreviousName_1CONDATE,PreviousName_1CompanyName,PreviousName_2CONDATE,PreviousName_2CompanyName,PreviousName_3CONDATE,PreviousName_3CompanyName,ConfStmtNextDueDate,ConfStmtLastMadeUpDate,nSIC,SIC1,SIC2,SIC3,eCompanyCategory,eCompanyStatus,eCountryOfOrigin,eAccountsAccountCategory,dIncorporationDate,dAccountsNextDueDate,dAccountsLastMadeUpDate,dReturnsNextDueDate,dReturnsLastMadeUpDate,dConfStmtNextDueDate,dConfStmtLastMadeUpDate,dPreviousName_1CONDATE,dPreviousName_2CONDATE,namechanged,namechanged2,pcd,oseast1m,osnrth1m,oa11,cty,ru11ind,oac11,lat,long,imd,country,oac1,oac2,imdu,CompanyStatus_T1,class_1
0,! LTD,08209948,NaN,NaN,METROHOUSE 57 PEPPER ROAD,HUNSLET,LEEDS,YORKSHIRE,NaN,LS10 2RU,Private Limited Company,Active,United Kingdom,NaN,11/09/2012,30.0,9.0,30/06/2018,30/09/2016,DORMANT,09/10/2016,11/09/2015,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,25/09/2019,11/09/2017,99999.0,99.0,9.0,99.0,15,2,182,3,2012.857923,2018.495890,2016.748634,2016.693989,2015.857534,2019.734247,2017.857534,NaN,NaN,0,0,LS102RU,431572.0,430988.0,E00057795,E99999999,A1,8A1,53.774278,-1.522430,5831.0,E,8,8A,5648.931384,Active,False
1,!NNOV8 LIMITED,11006939,NaN,NaN,C/O FRANK HIRTH 1ST FLOOR,236 GRAY'S INN ROAD,LONDON,NaN,UNITED KINGDOM,WC1X 8HB,Private Limited Company,Active,United Kingdom,NaN,11/10/2017,31.0,3.0,11/07/2019,NaN,NO ACCOUNTS FILED,08/11/2018,NaN,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,24/10/2019,NaN,62090.0,62.0,0.0,90.0,15,2,182,10,2017.860274,2019.852055,NaN,2018.610959,NaN,2019.813699,NaN,NaN,NaN,0,0,WC1X8HB,530883.0,182219.0,E00004531,E99999999,A1,3B2,51.523730,-0.114790,14970.0,E,3,3B,14502.572940,Active,False
2,!NSPIRED LTD,SC421617,NaN,NaN,26 POLMUIR ROAD,NaN,ABERDEEN,NaN,UNITED KINGDOM,AB11 7SY,Private Limited Company,Active,United Kingdom,NaN,11/04/2012,30.0,3.0,30/12/2018,30/03/2017,TOTAL EXEMPTION FULL,09/05/2017,11/04/2016,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,25/04/2020,11/04/2017,70229.0,70.0,2.0,29.0,15,2,182,13,2012.844262,2018.997260,2017.243836,2017.679452,2016.844262,2020.316940,2017.843836,NaN,NaN,0,0,AB117SY,393776.0,804937.0,S00090544,S99999999,1,2C3,57.135299,-2.104462,6772.0,S,2,2C,6772.000000,Active,False
3,!NVERTD DESIGNS LIMITED,09152972,NaN,NaN,55A HIGH STREET,NaN,SILSOE,BEDFORDSHIRE,NaN,MK45 4EW,Private Limited Company,Active,United Kingdom,NaN,30/07/2014,31.0,7.0,30/04/2019,31/07/2017,N/A,27/08/2016,30/07/2015,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,13/08/2020,30/07/2017,58190.0,58.0,1.0,90.0,15,2,182,9,2014.578082,2019.328767,2017.580822,2016.655738,2015.578082,2020.617486,2017.578082,NaN,NaN,0,0,MK454EW,508137.0,235898.0,E00088094,E99999999,F1,1C3,52.010987,-0.425843,29038.0,E,1,1C,28131.310156,Active,False
4,!OBAC UK LIMITED,07687209,NaN,NaN,ENDEAVOUR HOUSE,COOPERS END ROAD,STANSTED AIRPORT,NaN,NaN,CM24 1SJ,Private Limited Company,Active,United Kingdom,NaN,29/06/2011,29.0,12.0,29/09/2018,31/12/2016,SMALL,27/07/2017,29/06/2016,1,1,0,0,0,0,15/07/2011,!OBAC FITTINGS LIMITED,NaN,NaN,NaN,NaN,13/07/2020,29/06/2017,70229.0,70.0,2.0,29.0,15,2,182,12,2011.493151,2018.745205,2017.000000,2017.569863,2016.494536,2020.532787,2017.493151,2011.536986,NaN,1,0,CM241SJ,554894.0,222515.0,E00112246,E10000012,E1,5B3,51.879690,0.248905,17104.0,E,5,5B,16569.940385,Active,False


In [5]:
#keep only those companies whose T0 status is active
df = df[df['CompanyStatus_T0']=='Active']

In [6]:
features = df[['CompanyNumber', 'oa11',
        'CountryOfOrigin','MortgagesNumMortCharges',
       'MortgagesNumMortOutstanding', 'MortgagesNumMortPartSatisfied', 'LimitedPartnershipsNumGenPartners',
       'LimitedPartnershipsNumLimPartners',  'SIC1',
       'eCompanyCategory', 'eCompanyStatus',
       'eAccountsAccountCategory', 'dIncorporationDate',
       'dAccountsNextDueDate',
       'dReturnsNextDueDate','dConfStmtNextDueDate','namechanged', 'namechanged2', 'class_1']].copy()
#Remove the rows with missing data 
features.dropna(how='any',inplace=True)

In [7]:
#isolate target outcome - company credit status at time T1
target = np.array(df['class_1'])
features = features.drop('class_1', axis=1)

#Create new features(periods)
T0 = 2018+(31+28)/365
T1 = 2019+(31+28)/365
features['dAccountsOutstandingTime'] = T0-features['dAccountsNextDueDate']
features['dConfStmtOutstandingTime'] = T0-features['dConfStmtNextDueDate']
features['dReturnsOutstandingTime'] = T0-features['dReturnsNextDueDate']
features['OperatingTime'] = T1-features['dIncorporationDate']
features.loc[features['dAccountsOutstandingTime']<0, 'dAccountsOutstandingTime'] = 0 
features.loc[features['dConfStmtOutstandingTime']<0, 'dConfStmtOutstandingTime'] = 0 
features.loc[features['dReturnsOutstandingTime']<0, 'dReturnsOutstandingTime'] = 0
features.drop(['dAccountsNextDueDate','dConfStmtNextDueDate','dReturnsNextDueDate','dIncorporationDate'],axis = 1,inplace = True)
features.head()

,CompanyNumber,oa11,CountryOfOrigin,MortgagesNumMortCharges,MortgagesNumMortOutstanding,MortgagesNumMortPartSatisfied,LimitedPartnershipsNumGenPartners,LimitedPartnershipsNumLimPartners,SIC1,eCompanyCategory,eCompanyStatus,eAccountsAccountCategory,namechanged,namechanged2,dAccountsOutstandingTime,dConfStmtOutstandingTime,dReturnsOutstandingTime,OperatingTime
0,08209948,E00057795,United Kingdom,0,0,0,0,0,99.0,15,2,3,0,0,0.0,0.0,1.467655,6.303720
1,11006939,E00004531,United Kingdom,0,0,0,0,0,62.0,15,2,10,0,0,0.0,0.0,0.000000,1.301370
2,SC421617,S00090544,United Kingdom,0,0,0,0,0,70.0,15,2,13,0,0,0.0,0.0,0.482192,6.317382
3,09152972,E00088094,United Kingdom,0,0,0,0,0,58.0,15,2,9,0,0,0.0,0.0,1.505906,4.583562
4,07687209,E00112246,United Kingdom,1,1,0,0,0,70.0,15,2,12,1,0,0.0,0.0,0.591781,7.668493


In [14]:
features.drop(['dConfStmtOutstandingTime','CountryOfOrigin','LimitedPartnershipsNumGenPartners','LimitedPartnershipsNumLimPartners','eCompanyStatus'],axis = 1,inplace = True)

In [15]:
features

,CompanyNumber,oa11,MortgagesNumMortCharges,MortgagesNumMortOutstanding,MortgagesNumMortPartSatisfied,SIC1,eCompanyCategory,eAccountsAccountCategory,namechanged,namechanged2,dAccountsOutstandingTime,dReturnsOutstandingTime,OperatingTime
0,08209948,E00057795,0,0,0,99.0,15,3,0,0,0.0,1.467655,6.303720
1,11006939,E00004531,0,0,0,62.0,15,10,0,0,0.0,0.000000,1.301370
2,SC421617,S00090544,0,0,0,70.0,15,13,0,0,0.0,0.482192,6.317382
3,09152972,E00088094,0,0,0,58.0,15,9,0,0,0.0,1.505906,4.583562
4,07687209,E00112246,1,1,0,70.0,15,12,1,0,0.0,0.591781,7.668493
5,02714021,E00016232,0,0,0,90.0,15,3,0,0,0.0,0.479452,26.232682
6,10275228,E00116620,0,0,0,33.0,15,10,0,0,0.0,0.473973,2.227218
7,03811958,E00125512,0,0,0,70.0,15,3,1,0,0.0,1.527764,19.605479
8,06439541,E00157855,0,0,0,99.0,15,3,0,0,0.0,1.175305,11.252055
9,02943302,E00117358,0,0,0,98.0,15,3,0,0,0.0,0.594521,24.671233
